# Двоичное дерево
В двоичном дереве поиска:
 - каждая вершина может иметь (или не иметь) левого и правого ребенка;
 - каждая вершина, кроме корня, имеет родителя.
 
Для каждой вершины дерева мы храним:
- значение value
- левый ребенок left
- правый ребенок right
- родитель parent

Ключи в двоичном дереве поиска хранятся с соблюдением свойства упорядоченности:

Пусть $x$ - произвольная вершина двоичного дерева поиска. Если вершина $y$ находится в левом поддереве вершины $x$, то $y.key \ge x.key$ . Если $y$ находится в правом поддереве x, то $y.key \le x.key$ .

In [45]:
import random
from binarytree import build
import copy

# класс узла дерева
class MyNode:
    def __init__(self,value,parent=None,left=None,right=None):
        self._value = value
        self.parent = parent
        self.right = right
        self.left = left
        self.balance = None

    @property
    def value(self):
        if self._value != None:
            return int(self._value)
        else:
            return None

    @value.setter
    def value_set(self,value):
        if value != None:
            self._value = int(value)
        else:
            self._value = None

    @property
    def value_for_print(self):
        if self.balance!=None and self._value!=None:
            return f'{self._value}({self.balance})'
        else:
            return self._value

    def __repr__(self):
        return str(self.__dict__)

In [46]:
# класс дерева двоичного поиска
class BSTree():
    def __init__(self):
        self.root = None
        self.data = {}

    def __repr__(self):
        res = "root="+str(self.root)+"\n"
        for k,v in self.data.items():
            res += f'{k} => {v} \n'
        return res
    
    def find_by_key(self, key):
        if key in self.data.keys(): return self.data[key].value
        else: return None

    def build_from_list(self, l):
        for node in l:
            self.insert(node)

    def add_node(self, value, parent=None, left=None, right=None):
        # Имеем в виду, что для построения полного дерева будет
        # множество узлов со значением None, поэтому id нужно держать отдельно уникальными,
        # а не совпадающими с value. По сути это имитация указателей. 
        id = random.randint(0, 10 ** 2)
        while id in self.data.keys():
            id = random.randint(0, 2 ** 31 - 1)
        self.data[id] = MyNode(value,parent,left,right)
        return id

    def _delete_node(self, node):
        return self.data.pop(node, None)

    def insert(self,value):
        self.root = self.__insert(value,self.root, self.root)

    def __insert(self,value, root = None, parent=None):
        if root == None:
            # self.data.append(MyNode(value,parent,None,None))
            return self.add_node(value, parent, None, None)
        elif value < self.data[root].value:
            self.data[root].left = self.__insert(value, self.data[root].left, root)
        elif value > self.data[root].value:
            self.data[root].right = self.__insert(value, self.data[root].right, root)
        return root

    def print(self):
        print(self.__print(self.root,' '))

    def __print(self,root,deep):
        res = deep + str(self.data[root].value_for_print)+f' ({root})' + '\n'
        if self.data[root].left != None:
            res +=self.__print(self.data[root].left, deep+'-')
        if self.data[root].right != None:
            res += self.__print(self.data[root].right,deep+'+')
        return res

    def search(self,value):
        return self.__search(value,self.root)

    def __search(self,value,root):
        if root == None or value == self.data[root].value:
            return root
        if value < self.data[root].value:
            return self.__search(value,self.data[root].left)
        else:
            return self.__search(value,self.data[root].right)

    def minimum(self,return_index=False):
        return self.__minimum(self.root,return_index)

    def __minimum(self, root,return_index = False):
        if self.data[root].left == None:
            if return_index:
                return root
            else:
                return self.data[root].value
        else:
            return self.__minimum(self.data[root].left,return_index)

    def maximum(self,return_index=False):
        return self.__maximum(self.root,return_index)

    def __maximum(self, root,return_index = False):
        if self.data[root].right == None:
            if return_index:
                return root
            else:
                return self.data[root].value
        else:
            return self.__maximum(self.data[root].right,return_index)

    def find_node_by_value(self,value):
        res = list(key for key,node in self.data.items() if node.value==value)
        if len(res)!=1:
            return None
        else:
            return res[0]

    # горизонтальный обход с помощью очереди,
    def print_by_levels(self, full_value=False):
        """
        горизонтальный обход дерева. В случае avl дерева выводится с балансом вершин
        :full_value если True, то выводится строковое представление узла, не пригодное длоя сравнивания
        :return: список из значений узлов дерева
        """
        res = []
        top = self.root
        queue = []
        while True:
            if full_value:
                res.append(self.data[top].value_for_print)
            else:
                res.append(self.data[top].value)
            if self.data[top].left != None:
                queue.append(self.data[top].left)
            if self.data[top].right != None:
                queue.append(self.data[top].right)
            if len(queue)>0:
                top = queue.pop(0)
            else:
                break
        return res

    # достраивание дерева до полного, заполнение X пустых ячеек
    def print_by_levels_full(self):
        # создаем копию текущего дерева, нужна копия с информацией о балансе
        mystic = copy.deepcopy(self)
        # достраиваем его до полного None узлами
        height = self.height()
        mystic.__make_full_tree(mystic.root,height)
        # получаем его запись в горизонтальном обходе. Эта запись пригодна
        # для визуализации binary tree
        full = mystic.print_by_levels(full_value=True)
        return full

    # понятно выводим дерево с помощью сторонней библиотеки
    def print_nice(self):
        if len(self.data)>0:
            full = self.print_by_levels_full()
        else:
            full = [None]
        bt = build(full)
        print(bt)
        return None


    # создание узлов со значением None, чтобы дерево было полным
    def __make_full_tree(self,root,height):
        if height>0:
            if self.data[root].left == None:
                self.data[root].left = self.add_node(None, parent=root, left=None, right=None)
            if height > 1:
                self.__make_full_tree(self.data[root].left, height-1)
            if self.data[root].right == None:
                self.data[root].right = self.add_node(None, parent=root, left=None, right=None)
            if height > 1:
                self.__make_full_tree(self.data[root].right, height-1)


    # время работы O(h), где h - высота дерева
    def find_next(self,node):
        if self.data[node].right != None:
            return self.__minimum(self.data[node].right,return_index=True)
        else:
            # идем наверх налево, пока не получится свернуть направо
            top_node = self.data[node].parent
            while top_node!= None and node == self.data[top_node].right:
                node = top_node
                top_node = self.data[top_node].parent
            return top_node

    # удаление элемента с одним и меньше ребенком = просто возвращаем этого ребенка или none
    def __remove1child(self,node):
        if self.data[node].left == None and self.data[node].right != None:
            # return self.__delete_node(node).right
            self.data[self.data[node].right].parent=self.data[node].parent
            return self.data[node].right
        if self.data[node].right == None and self.data[node].left != None:
            # return self.__delete_node(node).left
            self.data[self.data[node].left].parent = self.data[node].parent
            return self.data[node].left
        if self.data[node].right == None and self.data[node].left == None:
            # self.__delete_node(node)
            return None

    # удаление элемента с двумя детьми - на место удаляемого сдвигаем минимальный справа
    def __remove2child(self,node):
        # находим минимальный элемент справа
        min_index = self.__minimum(self.data[node].right, return_index=True)
        # удаляем этот минимальный элемент
        self.data[node].right = self.__remove(min_index,self.data[node].right)
        # ставим на место удаляемого элемента минимальный элемент
        self.data[min_index].right = self.data[node].right
        self.data[min_index].left = self.data[node].left
        # self.__delete_node(node)
        return min_index


    def __remove(self, node, root):
        # идем в левое поддерево
        if self.data[root].value>self.data[node].value:
            self.data[root].left = self.__remove(node, self.data[root].left)
            return root
        # идем в правое поддерево
        elif self.data[root].value<self.data[node].value:
            self.data[root].right = self.__remove(node, self.data[root].right)
            return root
        # уже корень поддерева совпадает с удаляемым элементом
        else:
            if self.data[node].left == None or self.data[node].right == None:
                return self.__remove1child(node)
            else:
                return self.__remove2child(node)

    # удаляем узел node_index, перестраиваем дерево
    def remove(self, node_index):
        if node_index not in self.data.keys():
            return None
        self.root = self.__remove(node_index,self.root)
        self._delete_node(node_index)

    # возвращает высоту всего дерева     
    def height(self):
        if self.root==None:
            return 0
        elif len(self.data) == 1:
            return 1
        else:
            return self._height(self.root)

    def height_node(self,root):
        if self.root == None:
            return 0
        else:
            return self._height(root)

    def _height(self,root):
        """
        рекурсивная внутренняя функция
        возвращает высоту поддерева с корнем в узле с ключом root
        """
        if self.data[root].left == None and self.data[root].right == None:
            return 0
        else:
            left_height, right_height = 0,0
            # print(root)
            if self.data[root].left!=None:
                left_height = self._height(self.data[root].left)
            if self.data[root].right!=None:
                right_height = self._height(self.data[root].right)
            if left_height>right_height:
                return left_height+1
            else:
                return right_height+1

    # удаляет поддерево со значением value
    def remove_subtree_by_value(self, value):
        self.__remove_subtree_by_value(value,self.root)

    def __remove_subtree_by_value(self, value, root):
        if self.data[root].value < value:
            self.data[root].right = self.__remove_subtree_by_value(value,self.data[root].right)
            return root
        elif self.data[root].value > value:
            self.data[root].left = self.__remove_subtree_by_value(value,self.data[root].left)
            return root
        else:
            self.__remove_subtree_by_node(root)
            # возвращаем None, чтобы уничтожить ссылку на себя в left/right родительского узла
            return None
        pass

    # удаляем поддерево по id узла
    def __remove_subtree_by_node(self, root):
        if self.data[root].left != None:
            self.__remove_subtree_by_node(self.data[root].left)
        if self.data[root].right != None:
            self.__remove_subtree_by_node(self.data[root].right)
        self._delete_node(root)

In [47]:
print('Продемонстрируем двоичное дерево поиска из n элементов.')
b = BSTree()
# n=random.randint(1,20)
n = 10
values = [i for i in range(0,n)]
random.shuffle(values)
# values = [0, 2, 6, 4, 5, 8, 14, 7, 9, 1, 12, 3, 10, 13, 11]
print('Случайный массив чисел, они же - ключи дерева')
print(values)
for val in values:
    b.insert(val)
print('Строим дерево поиска из этих элементов.')
b.print_nice()

Продемонстрируем двоичное дерево поиска из n элементов.
Случайный массив чисел, они же - ключи дерева
[6, 2, 4, 3, 7, 8, 1, 0, 9, 5]
Строим дерево поиска из этих элементов.

      ______6
     /       \
    2__       7
   /   \       \
  1     4       8
 /     / \       \
0     3   5       9



In [48]:
print('Продемонстрируем нахождение высоты дерева поиска.')
print(b.height())

Продемонстрируем нахождение высоты дерева поиска.
3


In [49]:
print('Продемонстрируем нахождение минимального элемента в дереве поиска.')
print(b.minimum())
# print(b.minimum(return_index=True))
print('Продемонстрируем нахождение максимального элемента в дереве поиска.')
print(b.maximum())
# print(b.maximum(return_index=True))

Продемонстрируем нахождение минимального элемента в дереве поиска.
0
Продемонстрируем нахождение максимального элемента в дереве поиска.
9


In [50]:
print('Продемонстрируем нахождение следующего за заданным элементом в дереве поиска.')
element = input('Введите элемент, следующий за которым нужно найти')
element_id = b.search(int(element))
print(b.find_by_key(b.find_next(element_id)))

Продемонстрируем нахождение следующего за заданным элементом в дереве поиска.
Введите элемент, следующий за которым нужно найти4
5


In [51]:
print('Продемонстрируем удаление поддеревьев из дерева поиска.',
      'Если удаляем корень дерева, то все дерево исчезает.',
      'Вводите ключ корян поддерева или n, чтобы закончить процесс.')

b = BSTree()
n = 10
values = [i for i in range(0,n)]
random.shuffle(values)
for val in values:
    b.insert(val)
print('Строим дерево поиска из этих элементов.')
b.print_nice()

command = input('Введите номер корня поддерева для удаления либо n?')
while command != 'n':
    b.remove_subtree_by_value(int(command))
    print('Мы удалили ', command)
    b.print_nice()
    command = input('Введите номер корня поддерева для удаления либо n?')

Продемонстрируем удаление поддеревьев из дерева поиска. Если удаляем корень дерева, то все дерево исчезает. Вводите ключ корян поддерева или n, чтобы закончить процесс.
Строим дерево поиска из этих элементов.

      ____5__
     /       \
    2__       7__
   /   \     /   \
  1     4   6     9
 /     /         /
0     3         8

Введите номер корня поддерева для удаления либо n?4
Мы удалили  4

      5__
     /   \
    2     7__
   /     /   \
  1     6     9
 /           /
0           8

Введите номер корня поддерева для удаления либо n?7
Мы удалили  7

      5
     /
    2
   /
  1
 /
0

Введите номер корня поддерева для удаления либо n?5
Мы удалили  5
None
Введите номер корня поддерева для удаления либо n?n


## АВЛ - деревья
*АВЛ-дерево*, оно же *сбалансированное дерево поиска*, - двоичное дерево поиска, в котором для каждого узла высота
левого поддерева отличается от высоты правого поддерева не больше, чем на единицу
Отсюда следует, что если в дереве количество узлов равно $x$ и достаточно велико, то его высота будет равно $O(log(x))$

Для поддержания этого свойства дерева необходимы операции балансировки, которые описаны в классе как small_zig, big_zig, small_zag, big_zag.

In [52]:
# класс AVL дерева
class AVLTree(BSTree):
    def add_balance_info(self):
        if self.root != None:
            self._add_balance_info(self.root)
        else:
            print('Tree is empty!')

    def _add_balance_info(self, root):
        # print(self.data[root].value)
        right_height = -1
        if self.data[root].right!=None:
            self._add_balance_info(self.data[root].right)
            right_height = self.height_node(self.data[root].right)
        left_height = -1
        if self.data[root].left!=None:
            self._add_balance_info(self.data[root].left)
            left_height = self.height_node(self.data[root].left)
        # print(f'value={self.data[root].value} right height={right_height} left_height={left_height}')
        self.data[root].balance = right_height-left_height

    def small_zig(self,node):
        """
        Правый поворот проводится, если баланс вершины <-1 ,
        то есть, если баланс = правая_высота-левая_высота, то слева дерево длиннее.
        малый правый поворот. Баланс вершины node (b) -2, баланс его левого ребенка a 0.
        После поворота балансы будут равны -1 и 1 соответственно.
               ____________node(-2)_
             /                      \
         __a(0)_                   some2
        /       \
     some1      subtree

 После поворота:
          __a(1)____________
        /                  \
   some1             ___node(-1)_
                  /             \
            subtree            some2

        :return возвращаем новый корень
        """
        a = self.data[node].left
        subtree = self.data[a].right

        # если у node был родитель, то меняем ему потомка
        # это не нужно делать в рекурсивном случае, когда всегда node всегда совпадает с корнем дерева
        if self.data[node].parent != None:
            parent_index = self.data[node].parent
            if self.data[parent_index].left == node:
                self.data[parent_index].left = a
            else:
                self.data[parent_index].right = a

        # меняем местами node и а, a теперь на вершине
        self.data[a].right = node
        self.data[a].parent = self.data[node].parent
        self.data[node].parent = a
        # подвешиваем к node слева правое поддерево от a
        self.data[node].left = subtree
        # если поддерево не пусто, то меняем ему потомка
        if subtree != None:
            self.data[subtree].parent = node
        # если это был корень, то запоминаем новый корень
        if self.root == node:
            self.root = a
        return a

    def build_from_list(self, l, balanced = True):
        """
        Постройка дерева по значениям из списка
        :param l: список значений
        :param balanced: балансируем дерево в процессе построения. По умолчанию - нет.
        Строительство без балансировки нужно для демонстрации поворотов.
        """
        if balanced:
            for node in l:
                self.insert(node)
        else:
            for node in l:
                BSTree.insert(self,node)

    def big_zig(self,node):
        """
        Правый поворот делается, если баланс корня <-1.
        То есть, если баланс=правая_высота-левая_высота, то слева дерево  длиннее, чем справа.
               ___________________node(-2)__
             /                              \
         __a(1)______                      some2
        /            \
     some1           c(1)_
                  /       \
    ``subtree_c_left     subtree_c_right_

    После поворота:
                   _______c(0)____________
              /                            \
         __a(-1)_                  _______node(0)__
        /        \                /                \
   some1    subtree_c_left   subtree_c_right    some2


        большой правый поворот. Баланс узла node -2, узла а 1, узла с 1.
        После поворота будет 0, -1, 0 соответственно.
        """
        a = self.data[node].left
        c = self.data[a].right
        subtree_c_right = self.data[c].right
        subtree_c_left = self.data[c].left
        # ставим c на место node, node становится правым ребенком c, a становится левым ребенком c
        self.data[c].left = a
        self.data[a].parent = c
        self.data[c].right = node
        # меняем родителя с на того, что был у node
        self.data[c].parent = self.data[node].parent
        # если у node был родитель, то меняем ему потомка
        # это не нужно делать в рекурсивном случае, когда всегда node всегда совпадает с корнем дерева
        if self.data[node].parent != None:
            parent_index = self.data[node].parent
            if self.data[parent_index].left == node:
                self.data[parent_index].left = c
            else:
                self.data[parent_index].right = c
        self.data[node].parent = c
        # подвешиваем бывшие поддеревья c
        self.data[a].right = subtree_c_left
        if subtree_c_left!=None:
            self.data[subtree_c_left].parent = a
        self.data[node].left = subtree_c_right
        if subtree_c_right!=None:
            self.data[subtree_c_right].parent = node
        # если это был корень, то запоминаем новый корень
        if self.root == node:
            self.root = c
        return c

    def small_zag(self, node):
        """
        Левый поворот проводится, если баланс вершины >1 ,
        то есть, если баланс = правая_высота-левая_высота, то справа дерево длиннее.
        Малый левый поворот. Баланс вершины node (b) 2, баланс его правого ребенка a 0.
        После поворота балансы будут равны 1 и -1 соответственно.
               ____________node(2)_
             /                      \
         some2                    __a(0)_
                                /       \
                         subtree       some1

 После поворота:
          __a(-1)____________
        /                   \
   ____node(1)__             some1
  /             \
 some2        subtree

        :return возвращаем новый корень
        """
        a = self.data[node].right
        subtree = self.data[a].left

        # если у node был родитель, то меняем ему потомка
        # это не нужно делать в рекурсивном случае, когда всегда node всегда совпадает с корнем дерева
        if self.data[node].parent != None:
            parent_index = self.data[node].parent
            if self.data[parent_index].left == node:
                self.data[parent_index].left = a
            else:
                self.data[parent_index].right = a

        # меняем местами node и а, a теперь на вершине
        self.data[a].left = node
        self.data[a].parent = self.data[node].parent
        self.data[node].parent = a
        # подвешиваем к node слева правое поддерево от a
        self.data[node].right = subtree
        # если у поддерево не пусто, то меняем ему потомка
        if subtree != None:
            self.data[subtree].parent = node
        # если это был корень, то запоминаем новый корень
        if self.root == node:
            self.root = a
        return a

    def big_zag(self,node):
        """
        Левый поворот делается, если баланс корня >1.
        То есть, если баланс=правая_высота-левая_высота, то справа дерево  длиннее, чем слева.
               ___________________node(2)__
             /                              \
         some1                          __a(-1)__
                                       /       \
                                   c(-1)        some2
                                /       \
    `             `subtree_c_left     subtree_c_right

    После поворота:
                   _______c(0)____________
              /                            \
         node()_                  _______a() ______
        /        \                /                \
   some1    subtree_c_left   subtree_c_right    some2


        большой левый поворот. Баланс узла node 2, узла а -1, узла с -1.
        После поворота будет 0, 1, 0 соответственно.
        """
        a = self.data[node].right
        c = self.data[a].left
        subtree_c_right = self.data[c].right
        subtree_c_left = self.data[c].left
        # ставим c на место node, node становится Левым ребенком c, a становится правым ребенком c
        self.data[c].right = a
        self.data[a].parent = c
        self.data[c].left = node
        # меняем родителя с на того, что был у node
        self.data[c].parent = self.data[node].parent
        # если у node был родитель, то меняем ему потомка
        # это не нужно делать в рекурсивном случае, когда всегда node всегда совпадает с корнем дерева
        if self.data[node].parent != None:
            parent_index = self.data[node].parent
            if self.data[parent_index].left == node:
                self.data[parent_index].left = c
            else:
                self.data[parent_index].right = c
        self.data[node].parent = c
        # подвешиваем бывшие поддеревья c
        self.data[a].left = subtree_c_right
        if subtree_c_right!=None:
            self.data[subtree_c_right].parent = a
        self.data[node].right = subtree_c_left
        if subtree_c_left!=None:
            self.data[subtree_c_left].parent = node
        # если это был корень, то запоминаем новый корень
        if self.root == node:
            self.root = c
        return c

    def insert(self,value):
        """
        вставляет новое значение в дерево, контролируя сбалансированность
        """
        # super().insert(value)
        self.root = self._insert(value,self.root)

    def _insert(self,value, root = None, parent=None):
        # self.print_nice()
        if value != None:
            value = int(value)
        if root == None:
            new_node = self.add_node(value,parent,None,None)
            return new_node
        elif value < self.data[root].value:
            self.data[root].left = self._insert(value, self.data[root].left, root)
        elif value > self.data[root].value:
            self.data[root].right = self._insert(value, self.data[root].right, root)
        if root != None:
            self._add_balance_info(root)
            if self.data[root].balance <-1 or self.data[root].balance>1:
                # print(f'root {self.data[root].value} is unbalanced ({self.data[root].balance})')
                # print('rebalance it')
                root = self._rebalance(root)
                # self.print_nice()
        return root

    def _rebalance(self,root):
        # слева больше, делаем правый поворот
        if self.data[root].balance < -1:
            left_child = self.data[root].left
            if left_child != None and self.data[left_child].balance <= 0:
                # print('small zig')
                root = self.small_zig(root)
            elif self.data[left_child].balance == 1:
                # print('big_zig')
                root = self.big_zig(root)
            else:
                print('Error while zig')
        # справа больше, делаем левый поворот
        elif self.data[root].balance > 1:
            right_child = self.data[root].right
            if right_child != None and self.data[right_child].balance >= 0:
                # print('small zag')
                root = self.small_zag(root)
            elif self.data[right_child].balance == -1:
                # print('big zag')
                root = self.big_zag(root)
            else:
                print('Error while zag')
        return root

    def remove_node_by_value(self, value):
        """
        удаляет вершину по значению. Сначала находит это значение в дереве,
        потом удаляет узел и перестраивает дерево, чтобы оно осталось сбалансированным
        """
        self.__remove_node_by_value(value, self.root)

    def __remove_node_by_value(self, value, root):
        if self.data[root].value < value:
            self.data[root].right = self.__remove_node_by_value(value, self.data[root].right)
            return root
        elif self.data[root].value > value:
            self.data[root].left = self.__remove_node_by_value(value, self.data[root].left)
            return root
        else:
            print(f'Found {root}')
            # (0) если это единственная вершина, то просто удаляем ее и дерево пусто
            if root == self.root and len(self.data)==1:
                BSTree._delete_node(self,root)
                self.root = None
                return None

            # (1) если у вершины нет детей, это лист и просто удаляем
            if self.data[root].left == None and self.data[root].right == None:
                BSTree._delete_node(self, root)
                return None
            # (2) если у удаляемой вершины нет левого ребенка, то ее правый ребенок обязан быть листом.
            # Ставим его на место удаляемой root, возвращаем его индекс для родительской вершины
            elif self.data[root].left == None:
                right_child = self.data[root].right
                parent = self.data[root].parent
                self.data[right_child].parent = parent
                # print(self.data)
                BSTree._delete_node(self,root)
                return right_child
            # (3) в случае, если есть левая вершина
            else:
                left_subtree = self.data[root].left
                right_subtree = self.data[root].right

                # находим r - самую правую вершину в левом поддереве
                r = left_subtree
                while self.data[r].right!=None:
                    r = self.data[r].right
                r_parent = self.data[r].parent
                r_left_subtree = self.data[r].left
                # переносим индекс r в вершину root
                parent = self.data[root].parent
                self.data[r].parent = self.data[root].parent
                if self.data[parent].left == root:
                    self.data[parent].left = r
                elif self.data[parent].right == root:
                    self.data[parent].right = r
                # убираем вершину r (у нее нет правого ребенка, поэтому она либо лист,
                # либо имеет левого ребенка, являющегося листом (случай 2)
                # отцепляем r от его родителя
                self.data[r_parent].right = None
                # если есть левый ребенок, прицепляем к родителю r
                if r_left_subtree != None and r != left_subtree:
                    self.data[r_parent].right = r_left_subtree
                    self.data[r_left_subtree].parent = r_parent

                # прицепляем к r правое поддерево root. У r правого поддерева не могло быть
                if right_subtree!=None:
                    self.data[r].right = right_subtree
                    self.data[right_subtree].parent = r
                else:
                    self.data[r].right = None
                # прицепляем к r левое поддерево root, если оно не состоит только из r
                print(left_subtree , r)
                if left_subtree != r and left_subtree!=None:
                    self.data[r].left = left_subtree
                    self.data[left_subtree].parent = r
                else:
                    self.data[r].left = r_left_subtree
                    if r_left_subtree!=None:
                        self.data[r_left_subtree].parent = r_parent

                BSTree._delete_node(self, root)
                # поднимаемся к корню, начиная с нового родителя r, производя балансировку
                print(f'deleted, r={self.data[r]}')
                self._add_balance_info(r)
                if self.data[r].balance < -1 or self.data[r].balance > 1:
                    print('need to rebalance!')
                    print(self.data)
                    self.print_nice()
                    r = self._rebalance(r)
                return r

In [53]:
# класс для построения тестового дерева
class TestTree():
    def __init__(self,n=None):
        if n == None:
            n= random.randint(1, 20)
        self.values = [i for i in range(0, n)]
        random.shuffle(self.values)

    def getlist(self):
        return self.values

In [54]:
print('Продемонстрируем работу AVL дерева: ')
TestAVLTree.run_interactive_tree()

Продемонстрируем работу AVL дерева: 
insert/delete/exit? +/-/e
+
insert -> 10

10

insert/delete/exit? +/-/e
+
insert -> 6

   __10(-1)
  /
6(0)

insert/delete/exit? +/-/e
+7
Not legal operation! 
insert/delete/exit? +/-/e
+
insert -> 7

   __7(0)__
  /        \
6(1)      10(-2)

insert/delete/exit? +/-/e
+
insert -> 4

         __7(-1)__
        /         \
   __6(-1)       10(0)
  /
4(0)

insert/delete/exit? +/-/e
+
insert -> 5

        _______7(-1)__
       /              \
   __5(0)_           10(0)
  /       \
4(0)      6(0)

insert/delete/exit? +/-/e
+
insert -> 11

        _______7(0)__
       /             \
   __5(0)_          10(1)__
  /       \                \
4(0)      6(0)            11(0)

insert/delete/exit? +/-/e
-
delete -> 10
Found 100

        _______7(0)__
       /             \
   __5(0)_          11(0)
  /       \
4(0)      6(0)

insert/delete/exit? +/-/e
-
delete -> 5
Found 63
65 65
deleted, r={'_value': 4, 'parent': 16, 'right': 93, 'left': None, 'balance': 0}


In [55]:
# служебный класс для тестирования AVL дерева
class TestAVLTree():
    test_dict = {
        'small_zig': {'method': 'small_zig',
                      'values': [7, 3, 8, 2, 4, 1, 5],
                      'caption': 'ТЕСТИРУЕМ МАЛЫЙ ПРАВЫЙ ПОВОРОТ',
                      'information': """
       дерево для малого правого поворота.
       Баланс узла "7" -2, баланс узла "3" 0. Меняем их местами.
       Теперь баланс "7" -1, баланс "3" 1                  
                                   """
                      },
        'big_zig': {'method': 'big_zig',
                    'values': [10, 5, 14, 4, 7, 15, 3, 6, 8, 9],
                    'caption': 'ТЕСТИРУЕМ БОЛЬШОЙ ПРАВЫЙ ПОВОРОТ',
                    'information': """
       дерево для большого правого поворота
       Баланс узла "10" -2, узла "5" 1, узла "7" 1
       Теперь баланс узла "10" 0, "5" -1, "7" 0'                    
                           """
                    },
        'small_zag': {'method': 'small_zag',
                      'values': [7, 5, 12, 10, 15, 9, 16],
                      'caption': 'ТЕСТИРУЕМ МАЛЫЙ ЛЕВЫЙ ПОВОРОТ',
                      'information': """
       дерево для малого левого поворота
       Баланс узла "7" 2, баланс узла "12" 0. Меняем их местами.
       Теперь баланс "7" 1, баланс "12" -1                    
                               """
                      },
        'big_zag': {'method': 'big_zag',
                    'values': [10, 8, 15, 5, 9, 13, 18, 12, 14, 20, 11],
                    'caption': 'ТЕСТИРУЕМ БОЛЬШОЙ ЛЕВЫЙ ПОВОРОТ',
                    'information': """
       дерево для большого левого поворота
       Баланс узла "10" 2, узла "15" -1, узла "13" -1
       Теперь баланс узла "10" 0, "15" 1, "13" 0                 
                               """
                    }
    }

    @staticmethod
    def _run_zigzags(method, values, caption, information):
        print(caption.upper())
        print(information)
        print(values)
        t = AVLTree()
        t.build_from_list(values,balanced=False)
        t.add_balance_info()
        t.print_nice()
        # Тестируем метод
        getattr(t,method)(t.root)
        t.add_balance_info()
        t.print_nice()

    @classmethod
    def run_all_methods(cls):
        for case in cls.test_dict.values():
            cls._run_zigzags(case['method'],
                    case['values'],
                    case['caption'],
                    case['information'])

    @classmethod
    def run_inserts(cls,n,tree_size):
        for i in range(0, n):
            testvals = TestTree(tree_size).getlist()
            # testvals = [6, 9, 8, 4, 7, 5, 2, 3, 1, 0]
            try:
                print(i)
                print(testvals)
                t = AVLTree()
                t.build_from_list(testvals, balanced=True)
            except Exception as exc:
                print(f'Exception! with {testvals}', str(exc))
                break
            t.print_nice()

    @classmethod
    def run_interactive_tree(cls):
        t = AVLTree()
        while True:
            operation = input('insert/delete/exit? +/-/e\n')
            if operation == '+':
                item = int(input('insert -> '))
                t.insert(item)
                t.print_nice()
            elif operation == '-':
                item = int(input('delete -> '))
                t.remove_node_by_value(item)
                t.print_nice()
            elif operation == 'e':
                print('Exit')
                break
            else:
                print('Not legal operation! ')

    @classmethod
    def run_removes(cls):
        test_values = [
            [[10],[10]],
            [[1,0,8,10],[8]],
            [[3, 0, 2, 1, 4, 5], [0]],
            [[3,0,2,1,4,5],[5,3,1]],
            [[3,0,2,1,4,5],[4]],
            [[10,5,15,17,20,25,12,3,4],[15]],
            [[10, 9, 15, 17, 20, 25, 12, 3, 8], [10]],
            [[10, 9, 15, 17, 25, 12, 8,26,27,28,30,35], [27,26,10]]
        ]
        for i,case in enumerate(test_values):
            print(f'--------- test case {i} ----------------- ')
            t = AVLTree()
            for item in case[0]:
                t.insert(item)
            t.print_nice()
            for item in case[1]:
                print(f'delete {item}')
                t.remove_node_by_value(item)
                t.print_nice()

In [57]:
print('Проведем автоматическое тестирование класса AVL дерева.')
TestAVLTree.run_all_methods()
TestAVLTree.run_inserts(5,10)
TestAVLTree.run_removes()

Проведем автоматическое тестирование класса AVL дерева.
ТЕСТИРУЕМ МАЛЫЙ ПРАВЫЙ ПОВОРОТ

       дерево для малого правого поворота.
       Баланс узла "7" -2, баланс узла "3" 0. Меняем их местами.
       Теперь баланс "7" -1, баланс "3" 1                  
                                   
[7, 3, 8, 2, 4, 1, 5]

              ____________7(-2)_
             /                  \
         __3(0)_                8(0)
        /       \
   __2(-1)      4(1)_
  /                  \
1(0)                 5(0)


         __3(1)____________
        /                  \
   __2(-1)         _______7(-1)_
  /               /             \
1(0)            4(1)_           8(0)
                     \
                     5(0)

ТЕСТИРУЕМ БОЛЬШОЙ ПРАВЫЙ ПОВОРОТ

       дерево для большого правого поворота
       Баланс узла "10" -2, узла "5" 1, узла "7" 1
       Теперь баланс узла "10" 0, "5" -1, "7" 0'                    
                           
[10, 5, 14, 4, 7, 15, 3, 6, 8, 9]

              ____